In [1]:
import functools
import os
import pickle
import random

import numpy as np
import pandas as pd
import scipy.stats as st
import statsmodels.formula.api as smf


@functools.wraps(smf.ols)
def lm(*args, **kwargs):
    return smf.ols(*args, **kwargs).fit()

In [2]:
recommendee = "Fro116"
confidence_interval = 0.95 # x standard deviations
delta_sources = ["item", "user", "related", "maluserrec3"]
cross_validate = True  # if true, train linear model on out of sample data
renormalize_variance_iters = 10

In [3]:
outdir = f"../../data/recommendations/{recommendee}"
os.chdir(outdir)

In [4]:
def get_deltas(sources):
    deltas = []
    for source_filename in sources:
        delta = pickle.load(open(source_filename, "rb"))
        source = source_filename.split(".")[0].split("_")[0]
        delta = delta.rename({x: x + f"_{source}" for x in delta.columns}, axis=1)
        deltas.append(delta)
    return pd.concat(deltas, axis=1)

In [5]:
def clean_data(df):
    # fill missing data with reasonable defaults
    delta_sources = [x.split("_")[-1] for x in df.columns if "delta_var" in x]
    for source in delta_sources:
        df.loc[lambda x: x[f"delta_var_{source}"] == np.inf, f"delta_{source}"] = np.nan
        df.loc[
            lambda x: x[f"delta_var_{source}"] == np.inf, f"delta_var_{source}"
        ] = np.nan

        df[f"delta_{source}"] = df[f"delta_{source}"].fillna(0)
        df[f"delta_var_{source}"] = df[f"delta_var_{source}"].fillna(
            df[f"delta_var_{source}"].dropna().median()
        )
    return df

In [6]:
if cross_validate:
    train_df = get_deltas([f"{x}_loocv.pkl" for x in delta_sources])
else:
    train_df = get_deltas([f"{x}.pkl" for x in delta_sources])
delta_corrs = train_df[[f"delta_{source}" for source in delta_sources]].corr()

In [7]:
labelled_data = pickle.load(open("user_anime_list.pkl", "rb"))
labelled_data = clean_data(labelled_data.merge(train_df, on="anime_id", how='left'))

In [8]:
# get model
delta_cols = [f"delta_{source}" for source in delta_sources]
formula = "score ~ " + " + ".join(delta_cols)
model = lm(formula, labelled_data)
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  score   R-squared:                       0.274
Model:                            OLS   Adj. R-squared:                  0.265
Method:                 Least Squares   F-statistic:                     32.39
Date:                Wed, 26 May 2021   Prob (F-statistic):           6.39e-23
Time:                        22:03:59   Log-Likelihood:                -620.87
No. Observations:                 349   AIC:                             1252.
Df Residuals:                     344   BIC:                             1271.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.1671      0.11

In [9]:
#labelled_data[['score', 'delta_maluserrec', 'delta_item', 'delta_user']].corr()

In [10]:
df = clean_data(get_deltas([f"{x}.pkl" for x in delta_sources]))

In [11]:
blp = pickle.load(open("baseline_predictor.pkl", "rb"))
df["blp"] = blp["blp"]
df["score"] = model.predict(df) + df["blp"]
df["delta"] = df["score"] - df["blp"]

## renormalize the confidence intervals

In [12]:
for _ in range(renormalize_variance_iters):
    for source in delta_sources:
        seen_shows = pickle.load(open("user_anime_list.pkl", "rb"))
        seen_shows = seen_shows.set_index("anime_id")
        seen_shows["delta"] = df[f"delta_{source}"]

        single_delta_model = lm("score ~ delta + 0", seen_shows)

        seen_shows["pred_score"] = single_delta_model.predict(df)
        seen_shows["pred_std"] = np.sqrt(
            (df[f"delta_var_{source}"] + df[f"delta_{source}"] ** 2)
            * (
                single_delta_model.bse["delta"] ** 2
                + single_delta_model.params["delta"] ** 2
            )
            - (df[f"delta_{source}"] ** 2 * single_delta_model.params["delta"] ** 2)
        )
        seen_shows = seen_shows.loc[lambda x: x["pred_std"] < np.inf]

        std_mult = (
            (seen_shows["pred_score"] - seen_shows["score"]) / seen_shows["pred_std"]
        ).std()
        df[f"delta_var_{source}"] *= std_mult ** 2

In [13]:
# compute error bars
model_vars = pd.DataFrame()
for col in delta_cols:
    source = col.split("_")[1]
    model_vars[f"model_delta_var_{source}"] = (
        (df[f"delta_var_{source}"] + df[f"delta_{source}"] ** 2)
        * (model.bse[f"delta_{source}"] ** 2 + model.params[f"delta_{source}"] ** 2)
    ) - df[f"delta_{source}"] ** 2 * model.params[f"delta_{source}"] ** 2
model_stds = np.sqrt(model_vars)

delta_corrs = delta_corrs.loc[lambda x: (x.index.isin(delta_cols)), delta_cols]
delta_variance = np.sum(
    (model_stds.values @ delta_corrs.values) * model_stds.values, axis=1
)
intercept_variance = 0
if "Intercept" in model.bse:
    intercept_variance = model.bse["Intercept"] ** 2
df["std"] = np.sqrt(delta_variance + intercept_variance)

In [14]:
for _ in range(renormalize_variance_iters):
    seen_shows = pickle.load(open("user_anime_list.pkl", "rb"))
    seen_shows = seen_shows.set_index("anime_id")
    seen_shows["score"] += df["blp"]
    seen_shows["pred_score"] = df[f"score"]
    seen_shows["pred_std"] = df["std"]

    seen_shows = seen_shows.loc[lambda x: x["pred_std"] < np.inf]

    std_mult = (
        (seen_shows["pred_score"] - seen_shows["score"]) / seen_shows["pred_std"]
    ).std()
    df["std"] *= std_mult

In [15]:
zscore = st.norm.ppf(1 - (1 - confidence_interval) / 2)
df["score_lower_bound"] = df["score"] - df["std"] * zscore
df["score_upper_bound"] = df["score"] + df["std"] * zscore

In [16]:
anime = pd.read_csv("../../cleaned_data/anime.csv")
anime = anime[["anime_id", "title", "type"]]
df = df.merge(anime, on="anime_id").set_index("anime_id")

In [17]:
# reorder the columns
cols = [
    "title",
    "type",
    "score",
    "score_lower_bound",
    "score_upper_bound",
    "delta",
    "std",
] + delta_cols
df = df[cols + [x for x in df.columns if x not in cols]]

In [18]:
confidence_interval, zscore

(0.95, 1.959963984540054)

In [19]:
related_series = pickle.load(open("../../processed_data/related_anime_graph.pkl", "rb"))
df = df.merge(related_series, on="anime_id").set_index("anime_id")

In [20]:
new_recs = df.loc[lambda x: ~x.index.isin(labelled_data.anime_id) & (x["type"] == "TV")]

In [21]:
df.sort_values(by="score_lower_bound", ascending=False)[
    :20
]

,title,type,score,score_lower_bound,score_upper_bound,delta,std,delta_item,delta_user,delta_related,delta_maluserrec3,delta_var_item,delta_var_user,delta_var_related,delta_var_maluserrec3,blp,series_id
anime_id,,,,,,,,,,,,,,,,,
32,Neon Genesis Evangelion: The End of Evangelion,Movie,9.470785,7.748607,11.192962,2.189458,0.878678,-0.103332,1.092234,2.285116,1.044580,0.242022,0.416824,1.028549,1.145467,7.281326,20
5114,Fullmetal Alchemist: Brotherhood,TV,9.274007,7.503434,11.044580,1.171124,0.903370,-0.454362,-0.208816,0.000000,2.182521,0.157666,0.204073,1.620177,2.593584,8.102883,2738
11979,Mahou Shoujo Madoka★Magica Movie 2: Eien no Mo...,Movie,9.155442,7.445484,10.865400,1.796650,0.872444,-0.001944,0.240971,2.614022,1.079136,0.195951,0.617167,0.902768,1.401445,7.358792,4026
9756,Mahou Shoujo Madoka★Magica,TV,8.489900,7.360716,9.619083,1.084504,0.576125,-0.245452,0.421659,2.614022,0.136798,0.117351,0.260851,0.902768,0.212365,7.405396,4026
3784,Evangelion: 2.0 You Can (Not) Advance,Movie,9.012478,7.334388,10.690568,1.509456,0.856184,-0.106574,0.571642,2.300928,0.574129,0.234719,0.315450,0.569289,1.811907,7.503022,1701
30,Neon Genesis Evangelion,TV,8.447606,7.151313,9.743898,1.299163,0.661386,-0.062037,0.835813,2.285116,-0.040360,0.221711,0.320320,1.028549,0.188490,7.148442,20
11981,Mahou Shoujo Madoka★Magica Movie 3: Hangyaku n...,Movie,8.688120,7.102386,10.273853,1.321559,0.809063,-0.151492,0.513312,2.614022,0.283841,0.168799,0.484036,0.902768,1.124356,7.366560,4026
2759,Evangelion: 1.0 You Are (Not) Alone,Movie,8.768462,7.042964,10.493960,1.660344,0.880372,-0.021774,0.505513,2.300928,0.764221,0.233925,0.297051,0.569289,2.124862,7.108118,1701
7311,Suzumiya Haruhi no Shoushitsu,Movie,8.159952,7.028723,9.291181,0.385463,0.577168,-0.173952,0.031557,1.380163,-0.073321,0.121310,0.235886,0.598191,0.567034,7.774489,572


In [22]:
epsilon=1e-6
new_recs.loc[lambda x: (x["delta"] > model.params['Intercept'] + epsilon)].sort_values(
    by="score_lower_bound", ascending=False
).groupby("series_id").first().sort_values(by="score_lower_bound", ascending=False)[:50]

,title,type,score,score_lower_bound,score_upper_bound,delta,std,delta_item,delta_user,delta_related,delta_maluserrec3,delta_var_item,delta_var_user,delta_var_related,delta_var_maluserrec3,blp
series_id,,,,,,,,,,,,,,,,
2738,Fullmetal Alchemist: Brotherhood,TV,9.274007,7.503434,11.044580,1.171124,0.903370,-0.454362,-0.208816,0.000000,2.182521,0.157666,0.204073,1.620177,2.593584,8.102883
2690,Steins;Gate 0,TV,8.991758,6.473860,11.509657,1.092721,1.284666,-0.290486,-0.207602,0.988575,1.484362,0.580360,2.778572,1.626520,1.513163,7.899037
4084,Love Live! School Idol Project 2nd Season,TV,7.430610,5.437177,9.424043,0.548810,1.017076,0.161711,-0.112264,0.535378,0.269781,0.603449,1.000104,1.652147,1.210815,6.881801
156,Jigoku Shoujo Futakomori,TV,7.383730,5.406309,9.361152,0.374007,1.008907,-0.586243,-0.254810,0.000000,1.217716,0.297326,1.258492,1.620177,1.313681,7.009724
109,RahXephon,TV,6.878834,5.110444,8.647223,0.445470,0.902256,-0.115282,-0.033752,0.000000,0.568839,0.275476,1.522035,1.620177,0.403559,6.433364
124,Tenkuu no Escaflowne,TV,7.031785,5.071598,8.991972,0.381032,1.000114,-0.064500,0.019650,0.000000,0.364295,0.323571,1.545941,1.620177,0.988815,6.650753
1143,Hidamari Sketch x 365,TV,7.004389,5.055099,8.953679,0.177070,0.994554,0.023257,0.151508,0.000000,-0.163555,0.258761,2.027032,1.620177,0.603533,6.827319
7707,Kono Bijutsubu ni wa Mondai ga Aru!,TV,6.741276,4.982038,8.500514,0.469215,0.897587,0.069243,-0.072971,0.000000,0.434480,0.406927,1.037330,1.620177,0.623003,6.272061
51,Mobile Suit Zeta Gundam,TV,8.210964,4.931934,11.489994,1.342006,1.673005,-0.040197,0.172291,0.652239,1.319441,0.596725,3.788855,0.859624,6.352240,6.868958


In [26]:
epsilon=1e-6
new_recs.loc[lambda x: (x["delta"] < model.params['Intercept'] + epsilon)].sort_values(
    by="score_lower_bound", ascending=False
).groupby("series_id").first().sort_values(by="score_lower_bound", ascending=False)[:50]

,title,type,score,score_lower_bound,score_upper_bound,delta,std,delta_item,delta_user,delta_related,delta_maluserrec3,delta_var_item,delta_var_user,delta_var_related,delta_var_maluserrec3,blp
series_id,,,,,,,,,,,,,,,,
6375,Shirobako,TV,7.172108,5.817765,8.526451,-0.092736,0.691004,-0.203345,-0.077415,0.000000,-0.071065,0.136936,0.535066,1.620177,0.489407,7.264844
9,Monster,TV,7.117539,5.709688,8.525391,-0.487201,0.718305,-0.373457,-0.209448,0.000000,-0.320739,0.189960,0.561412,1.620177,0.449154,7.604740
3532,Bakuman. 2nd Season,TV,6.985169,5.691614,8.278724,-0.448092,0.659989,-0.536351,-0.265513,0.000000,-0.023880,0.191552,0.380663,1.620177,0.330737,7.433261
464,JoJo no Kimyou na Bouken: Diamond wa Kudakenai,TV,7.112249,5.658433,8.566066,-0.351275,0.741757,-0.445939,-0.075386,0.000000,-0.175517,0.210627,0.857334,1.620177,0.226474,7.463524
2399,Natsume Yuujinchou Shi,TV,7.068159,5.637667,8.498650,-0.646757,0.729856,-0.583754,-0.472809,0.000000,-0.051744,0.169567,0.587517,1.620177,0.421702,7.714916
5183,Non Non Biyori Repeat,TV,7.137369,5.618281,8.656457,-0.007022,0.775059,-0.142463,-0.027089,0.103276,-0.107051,0.232925,0.755211,1.562565,0.518367,7.144391
5615,Haikyuu!!: Karasuno Koukou VS Shiratorizawa Ga...,TV,7.101677,5.605069,8.598285,-0.735676,0.763590,-0.433360,-0.298338,0.507295,-0.729815,0.253699,0.564980,1.483549,0.565312,7.837353
607,Gintama&#039;: Enchousen,TV,7.080235,5.571135,8.589334,-0.893982,0.769963,-0.627629,-0.289130,0.000000,-0.545933,0.196238,0.818578,1.620177,0.357591,7.974216
936,Kanon (2006),TV,6.830957,5.502248,8.159667,-0.325880,0.677925,-0.182256,-0.184243,0.000000,-0.327022,0.172516,0.478721,1.620177,0.395847,7.156838


In [23]:
new_recs.loc[lambda x: (x["delta"] <0)].sort_values(
    by="score_lower_bound", ascending=False
).groupby("series_id").first().sort_values(by="score_lower_bound", ascending=False)[:50]

,title,type,score,score_lower_bound,score_upper_bound,delta,std,delta_item,delta_user,delta_related,delta_maluserrec3,delta_var_item,delta_var_user,delta_var_related,delta_var_maluserrec3,blp
series_id,,,,,,,,,,,,,,,,
6375,Shirobako,TV,7.172108,5.817765,8.526451,-0.092736,0.691004,-0.203345,-0.077415,0.000000,-0.071065,0.136936,0.535066,1.620177,0.489407,7.264844
9,Monster,TV,7.117539,5.709688,8.525391,-0.487201,0.718305,-0.373457,-0.209448,0.000000,-0.320739,0.189960,0.561412,1.620177,0.449154,7.604740
3532,Bakuman. 2nd Season,TV,6.985169,5.691614,8.278724,-0.448092,0.659989,-0.536351,-0.265513,0.000000,-0.023880,0.191552,0.380663,1.620177,0.330737,7.433261
464,JoJo no Kimyou na Bouken: Diamond wa Kudakenai,TV,7.112249,5.658433,8.566066,-0.351275,0.741757,-0.445939,-0.075386,0.000000,-0.175517,0.210627,0.857334,1.620177,0.226474,7.463524
2399,Natsume Yuujinchou Shi,TV,7.068159,5.637667,8.498650,-0.646757,0.729856,-0.583754,-0.472809,0.000000,-0.051744,0.169567,0.587517,1.620177,0.421702,7.714916
5183,Non Non Biyori Repeat,TV,7.137369,5.618281,8.656457,-0.007022,0.775059,-0.142463,-0.027089,0.103276,-0.107051,0.232925,0.755211,1.562565,0.518367,7.144391
5615,Haikyuu!!: Karasuno Koukou VS Shiratorizawa Ga...,TV,7.101677,5.605069,8.598285,-0.735676,0.763590,-0.433360,-0.298338,0.507295,-0.729815,0.253699,0.564980,1.483549,0.565312,7.837353
607,Gintama&#039;: Enchousen,TV,7.080235,5.571135,8.589334,-0.893982,0.769963,-0.627629,-0.289130,0.000000,-0.545933,0.196238,0.818578,1.620177,0.357591,7.974216
936,Kanon (2006),TV,6.830957,5.502248,8.159667,-0.325880,0.677925,-0.182256,-0.184243,0.000000,-0.327022,0.172516,0.478721,1.620177,0.395847,7.156838


In [24]:
new_recs.loc[lambda x: (x["delta_item"] < 0) & (x["delta_user"] > 0)].sort_values(
    by="score_lower_bound", ascending=False
).groupby("series_id").first().sort_values(by="score_lower_bound", ascending=False)[:50]

,title,type,score,score_lower_bound,score_upper_bound,delta,std,delta_item,delta_user,delta_related,delta_maluserrec3,delta_var_item,delta_var_user,delta_var_related,delta_var_maluserrec3,blp
series_id,,,,,,,,,,,,,,,,
124,Tenkuu no Escaflowne,TV,7.031785,5.071598,8.991972,0.381032,1.000114,-0.064500,0.019650,0.000000,0.364295,0.323571,1.545941,1.620177,0.988815,6.650753
51,Mobile Suit Zeta Gundam,TV,8.210964,4.931934,11.489994,1.342006,1.673005,-0.040197,0.172291,0.652239,1.319441,0.596725,3.788855,0.859624,6.352240,6.868958
1143,Hidamari Sketch,TV,6.661025,4.921932,8.400118,0.309294,0.887308,-0.003216,0.152487,0.000000,0.057699,0.242313,1.558971,1.620177,0.359151,6.351731
91,Kimi ga Nozomu Eien,TV,6.510459,4.838508,8.182411,0.190382,0.853052,-0.048550,0.086295,0.000000,0.002163,0.317018,1.050995,1.620177,0.490951,6.320078
7657,Amanchu!,TV,6.695332,4.657520,8.733145,0.459486,1.039720,-0.043518,0.233290,0.000000,0.240724,0.237943,2.141247,1.620177,0.879691,6.235846
894,Maison Ikkoku,TV,7.139748,4.330890,9.948606,0.082422,1.433117,-0.091406,0.051587,0.000000,-0.071777,0.784015,4.773073,1.620177,0.915789,7.057327
7908,Uchuu Patrol Luluco,TV,6.158109,4.295811,8.020407,-0.178114,0.950170,-0.397821,0.248551,0.000000,-0.302422,0.443289,1.343795,1.620177,0.546637,6.336223
586,Digimon Tamers,TV,6.376305,4.288705,8.463905,0.023712,1.065121,-0.129136,0.017756,0.000000,-0.080727,0.879122,1.665901,1.620177,0.437147,6.352593
2714,Kiss x Sis (TV),TV,5.841039,4.234320,7.447757,-0.011109,0.819769,-0.241703,0.011412,0.000000,0.001978,0.599460,0.691352,1.620177,0.268445,5.852147


In [25]:
new_recs.loc[lambda x: (x["delta_item"] > 0) & (x["delta_user"] < 0)].sort_values(
    by="score_lower_bound", ascending=False
).groupby("series_id").first().sort_values(by="score_lower_bound", ascending=False)[:20]

,title,type,score,score_lower_bound,score_upper_bound,delta,std,delta_item,delta_user,delta_related,delta_maluserrec3,delta_var_item,delta_var_user,delta_var_related,delta_var_maluserrec3,blp
series_id,,,,,,,,,,,,,,,,
4084,Love Live! School Idol Project 2nd Season,TV,7.430610,5.437177,9.424043,0.548810,1.017076,0.161711,-0.112264,0.535378,0.269781,0.603449,1.000104,1.652147,1.210815,6.881801
7708,New Game!!,TV,6.786450,5.017096,8.555804,-0.023281,0.902748,0.125980,-0.055330,0.388557,-0.520140,0.295563,1.342889,1.505768,0.564451,6.809731
7707,Kono Bijutsubu ni wa Mondai ga Aru!,TV,6.741276,4.982038,8.500514,0.469215,0.897587,0.069243,-0.072971,0.000000,0.434480,0.406927,1.037330,1.620177,0.623003,6.272061
5698,Gochuumon wa Usagi desu ka??,TV,6.898832,4.957067,8.840598,0.057269,0.990715,0.155319,-0.022046,0.000000,-0.312975,0.380112,1.624859,1.620177,0.704588,6.841564
9175,Karakai Jouzu no Takagi-san,TV,7.124049,4.806911,9.441186,0.529787,1.182235,0.030351,-0.147511,0.000000,0.641161,0.467411,1.465879,1.620177,2.542018,6.594262
3569,B-gata H-kei,TV,6.375093,4.804206,7.945980,0.306537,0.801488,0.005422,-0.041692,0.000000,0.238479,0.424209,0.636339,1.620177,0.465636,6.068556
1842,Bamboo Blade,TV,6.386560,4.580515,8.192605,-0.027056,0.921468,0.130664,-0.051401,0.000000,-0.378429,0.355691,1.528057,1.620177,0.380103,6.413616
792,Bokura ga Ita,TV,6.488040,4.559590,8.416491,0.080425,0.983921,0.028208,-0.039156,0.000000,-0.118729,0.438561,1.667697,1.620177,0.518442,6.407615
4094,Ikoku Meiro no Croisée The Animation,TV,6.592728,4.529201,8.656254,0.240812,1.052839,0.080156,-0.256492,0.000000,0.273736,0.851379,1.504842,1.620177,0.516600,6.351915
